In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 11.1 MB/s eta 0:00:00


In [3]:
from datasets.dataset_dict import DatasetDict
from datasets import Dataset

In [4]:
from datasets import load_dataset
#load the dataset
dataset = load_dataset("rahular/itihasa")
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 75162
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 6149
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 11722
    })
})

In [5]:
print(dataset['train'][:5])

{'translation': [{'en': 'The ascetic Vālmīki asked Nārada, the best of sages and foremost of those conversant with words, ever engaged in austerities and Vedic studies.', 'sn': 'ॐ तपः स्वाध्यायनिरतं तपस्वी वाग्विदां वरम्। नारदं परिपप्रच्छ वाल्मीकिर्मुनिपुङ्गवम्॥'}, {'en': 'Who at present in this world is like crowned with qualities, and with prowess, knowing duty, and grateful, and truthful, and firm in vow.', 'sn': 'कोन्वस्मिन् साम्प्रतं लोके गुणवान् कश्च वीर्यवान्। धर्मज्ञश्च कृतज्ञश्च सत्यवाक्यो दृढत्नतः॥'}, {'en': 'Who is qualified by virtue of his character, and who is engaged in the welfare of all creatures? Who is learned and capable. Who alone is ever lovely to behold?', 'sn': 'चारित्रेण च को युक्तः सर्वभूतेषु को हितः। विद्वान् कः कः समर्थश्च कश्चैकप्रियदर्शनः॥'}, {'en': 'Who has subdued his heart, and controlled his anger, is endowed with splendour, and devoid of malice; and whom enraged in battle, do even the gods, fear?', 'sn': 'आत्मवान् को जितक्रोधो द्युतिमान् कोऽनसूयकः। कस

In [6]:
print(dataset["train"][378])

{'translation': {'en': 'Slaying that one of terrible prowess, who distresses the three worlds, with his forces and friends, do you, O foremost of gods, your fever of heart removed, repair to the celestial regions protected by you and purged of all its faults and sins.', 'sn': 'तमेव हत्वा सबलं सबान्धवं विरावणं रावणमुग्रपौरुषम्। स्वर्लोकमागच्छ गतज्वरश्चिरं सुरेन्द्रगुप्तं गतदोषकल्मषम्॥'}}


In [7]:
len(dataset)

3

In [8]:
np.shape(dataset)

{'train': (75162, 1), 'validation': (6149, 1), 'test': (11722, 1)}

In [9]:
df = pd.DataFrame.from_dict(dataset, orient='index')

TEXTVECTORIZATION

In [10]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Extracting English and Sanskrit sentences from the dataset
english_sentences = [example["translation"]["en"] for example in dataset["train"]]
sanskrit_sentences = [example["translation"]["sn"] for example in dataset["train"]]

# Define text vectorization parameters
vocab_size = 10000
sequence_length = 30

# English text vectorization
english_vectorizer = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
english_vectorizer.adapt(english_sentences)

# Sanskrit text vectorization
sanskrit_vectorizer = TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
sanskrit_vectorizer.adapt(sanskrit_sentences)

# Check the vocabulary size for both languages
print("English vocabulary size:", len(english_vectorizer.get_vocabulary()))
print("Sanskrit vocabulary size:", len(sanskrit_vectorizer.get_vocabulary()))

English vocabulary size: 10000
Sanskrit vocabulary size: 10000


In [11]:
batch_size = 64

Formatting and making dataset

In [12]:
import tensorflow as tf

def format_dataset(english_texts, sanskrit_texts):
    # Vectorize English and Sanskrit sentences
    eng_vectorized = english_vectorizer(english_texts)
    sanskrit_vectorized = sanskrit_vectorizer(sanskrit_texts)

    # Prepare the inputs and targets for the model
    encoder_inputs = tf.expand_dims(eng_vectorized, axis=-1)
    decoder_inputs = tf.expand_dims(sanskrit_vectorized[:, :-1], axis=-1)
    targets = tf.expand_dims(sanskrit_vectorized[:, 1:], axis=-1)

    return {"encoder_inputs": encoder_inputs, "decoder_inputs": decoder_inputs}, targets

def make_dataset(dataset):
    eng_texts = [example["en"] for example in dataset["translation"]]
    sanskrit_texts = [example["sn"] for example in dataset["translation"]]

    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, sanskrit_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(dataset["train"])
val_ds = make_dataset(dataset["validation"])


*** BUILDING MODEL...***

***TRAINING MODEL with epochs***

In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Define MultiHeadAttention layer
class MultiHeadAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads, **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embed_dim must be divisible by num_heads. (embed_dim: {embed_dim}, num_heads: {num_heads})"
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs["query"], inputs["key"], inputs["value"], inputs["attention_mask"]
        batch_size = tf.shape(query)[0]

        # Linear projections
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # Separate heads
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)

        # Applying attention
        attention_output, _ = self.attention(query, key, value)
        attention_output = tf.transpose(attention_output, perm=[0, 2, 1, 3])

        # Concatenate heads
        concat_attention = tf.reshape(attention_output, (batch_size, -1, self.embed_dim))
        output = self.combine_heads(concat_attention)

        return output


# Define PositionalEmbedding layer
class PositionalEmbedding(layers.Layer):
    def __init__(self, vocab_size, embed_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=embed_dim, output_dim=embed_dim)

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(inputs)
        return x + positions


# Define TransformerEncoder layer
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention = MultiHeadAttention(embed_dim, num_heads)
        self.ffn = self.point_wise_feed_forward_network(latent_dim, embed_dim)
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(0.1)
        self.dropout2 = layers.Dropout(0.1)

    def point_wise_feed_forward_network(self, latent_dim, embed_dim):
        return keras.Sequential(
            [
                layers.Dense(latent_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.math.logical_not(mask)
        else:
            padding_mask = None

        attention_output = self.attention({
            "query": inputs,
            "key": inputs,
            "value": inputs,
            "attention_mask": padding_mask,
        })
        attention_output = self.dropout1(attention_output)
        x1 = self.layernorm1(inputs + attention_output)
        ffn_output = self.ffn(x1)
        ffn_output = self.dropout2(ffn_output)
        encoder_outputs = self.layernorm2(x1 + ffn_output)
        return encoder_outputs


# Define TransformerDecoder layer
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.mha1 = MultiHeadAttention(embed_dim, num_heads)
        self.mha2 = MultiHeadAttention(embed_dim, num_heads)
        self.ffn = self.point_wise_feed_forward_network(latent_dim, embed_dim)
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(0.1)
        self.dropout2 = layers.Dropout(0.1)
        self.dropout3 = layers.Dropout(0.1)

    def point_wise_feed_forward_network(self, latent_dim, embed_dim):
        return keras.Sequential(
            [
                layers.Dense(latent_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )

    def causal_attention_mask(self, batch_size, n_dest, n_src, dtype):
        i = tf.range(n_dest)[:, tf.newaxis]
        j = tf.range(n_src)
        m = i >= j - n_src + n_dest
        return tf.cast(m, dtype)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        sequence_length = input_shape[1]
        causal_mask = self.causal_attention_mask(batch_size, sequence_length, sequence_length, tf.float32)
        return causal_mask

    def call(self, inputs, encoder_outputs, mask=None):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        sequence_length = input_shape[1]

        causal_mask = self.get_causal_attention_mask(inputs)

        if mask is not None:
            padding_mask = tf.math.logical_not(mask)
        else:
            padding_mask = None

        attention_output_1 = self.mha1(
            {"query": inputs, "key": inputs, "value": inputs, "attention_mask": causal_mask}
        )
        attention_output_1 = self.dropout1(attention_output_1)
        out1 = self.layernorm1(inputs + attention_output_1)

        attention_output_2 = self.mha2(
            {"query": out1, "key": encoder_outputs, "value": encoder_outputs, "attention_mask": padding_mask}
        )
        attention_output_2 = self.dropout2(attention_output_2)
        out2 = self.layernorm2(out1 + attention_output_2)

        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output)
        decoder_outputs = self.layernorm3(out2 + ffn_output)

        return decoder_outputs


vocab_size = 10000
embed_dim = 256
latent_dim = 2048
num_heads = 8

sequence_length = 30

# Encoder
encoder_inputs = tf.keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = tf.keras.Model(encoder_inputs, encoder_outputs)

# Decoder
decoder_inputs = tf.keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = tf.keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs, mask=sanskrit_vectorizer.compute_mask(decoder_inputs))
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = tf.keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = tf.keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

# Compile and train the model
epochs = 10
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)


Epoch 1/10
1175/1175 [==============================] - 2031s 2s/step - loss: 1.8710 - accuracy: 0.7861 - val_loss: 1.6945 - val_accuracy: 0.7927
Epoch 2/10
1175/1175 [==============================] - 2007s 2s/step - loss: 1.6573 - accuracy: 0.7996 - val_loss: 1.3922 - val_accuracy: 0.8200
Epoch 3/10
1175/1175 [==============================] - 2007s 2s/step - loss: 0.9099 - accuracy: 0.8824 - val_loss: 0.4499 - val_accuracy: 0.9403
Epoch 4/10
1175/1175 [==============================] - 2005s 2s/step - loss: 0.3298 - accuracy: 0.9577 - val_loss: 0.1424 - val_accuracy: 0.9879
Epoch 5/10
1175/1175 [==============================] - 1998s 2s/step - loss: 0.1152 - accuracy: 0.9888 - val_loss: 0.0369 - val_accuracy: 0.9980
Epoch 6/10
1175/1175 [==============================] - 2005s 2s/step - loss: 0.0382 - accuracy: 0.9974 - val_loss: 0.0137 - val_accuracy: 0.9986
Epoch 7/10
1175/1175 [==============================] - 2009s 2s/step - loss: 0.0181 - accuracy: 0.9985 - val_loss: 0.0112 -

 **Decoding Test Sentence**

In [2]:
# Extract English and Sanskrit sentences from the test split of the dataset
test_eng_sentences = [example["translation"]["en"] for example in dataset["validation"]]
test_sanskrit_sentences = [example["translation"]["sn"] for example in dataset["validation"]]

# Create a list of English-Sanskrit sentence pairs for testing
test_pairs = list(zip(test_eng_sentences, test_sanskrit_sentences))

NameError: ignored

In [32]:
# for _ in range(10):
#     input_sentence = random.choice(test_eng_texts)
#     translated = decode_sequence(input_sentence)
#     print("Input Sentence:", input_sentence)
#     print("Translated Sentence:", translated)
#     print("Sampled Token Index:", sampled_token_index)
#     print("English Index Lookup:", english_index_lookup)
#     print()


KeyError: ignored

In [48]:
import random

max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = english_vectorizer([input_sentence])
    decoded_sentence = "[start]"
   # Inside the decode_sequence function
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = english_vectorizer([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])

        if sampled_token_index not in english_index_lookup:
            print("Sampled token index not found in the vocabulary:", sampled_token_index)

        sampled_token = english_index_lookup.get(sampled_token_index, "[unknown]")  # Use .get() to handle KeyError
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break



test_eng_texts = [pair[0] for pair in test_pairs]
english_index_lookup = dict(zip(english_vectorizer.get_vocabulary(), range(len(english_vectorizer.get_vocabulary()))))

# Generate and print translations for 10 random sentences from the test set
for _ in range(10):
    input_sentence = random.choice(test_eng_texts)
    print("Input Sentence:", input_sentence)
    translated = decode_sequence(input_sentence)
    print("Translated Sentence:", translated)
    print()


Input Sentence: We had been greatly tired. Our penances had einaciated us. At that time, a bodiless Being addressed us from the sky and said to us
Sampled token index not found in the vocabulary: 0
Sampled token index not found in the vocabulary: 0
Sampled token index not found in the vocabulary: 0
Sampled token index not found in the vocabulary: 0
Sampled token index not found in the vocabulary: 0
Sampled token index not found in the vocabulary: 0
Sampled token index not found in the vocabulary: 0
Sampled token index not found in the vocabulary: 0
Sampled token index not found in the vocabulary: 0
Sampled token index not found in the vocabulary: 0
Sampled token index not found in the vocabulary: 0
Sampled token index not found in the vocabulary: 0
Sampled token index not found in the vocabulary: 0
Sampled token index not found in the vocabulary: 0
Sampled token index not found in the vocabulary: 0
Sampled token index not found in the vocabulary: 0
Sampled token index not found in the 

In [1]:
import random

max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = english_vectorizer([input_sentence])
    decoded_sentence = "[start]"

    for i in range(max_decoded_sentence_length):
      tokenized_target_sentence = english_vectorizer([decoded_sentence])[:, :-1]  # Use English vectorizer here
      predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

      sampled_token_index = np.argmax(predictions[0, i, :])
      sampled_token = english_index_lookup[sampled_token_index]  # Use english_index_lookup
      decoded_sentence += " " + sampled_token

      if sampled_token == "[end]":
          break
      return decoded_sentence


test_eng_texts = [pair[0] for pair in test_pairs]
english_index_lookup = dict(zip(english_vectorizer.get_vocabulary(), range(len(english_vectorizer.get_vocabulary()))))

# Generate and print translations for 10 random sentences from the test set
for _ in range(10):
    input_sentence = random.choice(test_eng_texts)
    print("Input Sentence:", input_sentence)
    translated = decode_sequence(input_sentence)
    print("Translated Sentence:", translated)
    print()


NameError: ignored

**Implementing BLUE SCORE**

In [ ]:
reference_list = [['', '', ''], ['reference', 'sentence', '2'], ...]
candidate_list = [['candidate', 'sentence', '1'], ['candidate', 'sentence', '2'], ...]
##ref- like->'ॐ तपः स्वाध्यायनिरतं तपस्वी वाग्विदां वरम्। नारदं परिपप्रच्छ वाल्मीकिर्मुनिपुङ्गवम्

# Calculate BLEU score for each candidate sentence
bleu_scores = []
for candidate, references in zip(candidate_list, reference_list):
    candidate_tokens = candidate
    reference_tokens_list = references
    bleu = sentence_bleu(reference_tokens_list, candidate_tokens)
    bleu_scores.append(bleu)

# Calculate average BLEU score for the entire dataset
average_bleu_score = sum(bleu_scores) / len(bleu_scores)
print("Average BLEU Score:", average_bleu_score)
